# AI Trivia Contest — Gradio UI

An interactive UI that lets a judge AI pick domains and hints while three AI players compete to guess the word. Built on top of the existing `AITriviaContest` game logic with Ollama models.


## Install required models

Run once to ensure the models are available locally (idempotent):


In [ ]:
!ollama pull llama3.1
!ollama pull mistral:latest
!ollama pull phi3:latest
!ollama pull gemma3:270m


In [21]:
import gradio as gr
from ai_trivia_contest import AITriviaContest

# For reload during development
import importlib
import ai_trivia_contest
importlib.reload(ai_trivia_contest)
from ai_trivia_contest import AITriviaContest


In [22]:
# Configuration
judge_model = "llama3.1"
player_models = ["mistral:latest", "phi3:latest", "gemma3:270m"]
target_score = 100
points_per_answer = 10
max_turns = 10  # set None for no limit

print("✓ Config loaded")


✓ Config loaded


In [23]:
# Initialize game

game = AITriviaContest(
    judge_model=judge_model,
    player_models=player_models,
    target_score=target_score,
    points_per_answer=points_per_answer,
    max_turns=max_turns
)

print("Game initialized")


Game initialized


In [29]:
# Helper functions for UI formatting

PLAYER_SHAPES = ["🟢", "🔷", "🟡"]


def build_guess_lines(turn_result, game):
    lines = []
    guesses = turn_result.get("guesses", {}) if turn_result else {}
    correct_players = set(turn_result.get("evaluation", {}).get("correct_players", [])) if turn_result else set()
    if not guesses:
        return ""
    for idx, player in enumerate(game.player_models):
        shape = PLAYER_SHAPES[idx % len(PLAYER_SHAPES)]
        guess = guesses.get(player, "-")
        status = "✨✅" if player in correct_players else "❌"
        lines.append(f"{shape} `{player}` guessed `{guess}` {status}")
    return "\n".join(lines)


def format_status(game):
    turns_remaining = None
    if game.max_turns is not None:
        turns_remaining = max(game.max_turns - game.turn_number, 0)
    status_lines = [
        f"**Judge**: `{getattr(game, 'judge_model', 'judge')}`",
        f"**Players**: {', '.join([f'`{p}`' for p in game.player_models])}",
        f"**Target Score**: {game.target_score}",
        f"**Points/Correct**: {game.points_per_answer}",
    ]
    if game.max_turns is not None:
        status_lines.append(f"**Max Turns**: {game.max_turns} | **Turns Remaining**: {turns_remaining}")
    status_lines.append(f"**Turn #**: {game.turn_number}")
    status_lines.append(f"**Game Over**: {game.game_over}")
    if game.game_over and getattr(game, "winners", []):
        status_lines.append(f"**Winners**: {', '.join([f'`{w}`' for w in game.winners])}")
    return "\n".join(status_lines)


def build_guesses_table(turn_result):
    rows = []
    guesses = turn_result.get("guesses", {})
    correct_players = set(turn_result.get("evaluation", {}).get("correct_players", []))
    for player, guess in guesses.items():
        status = "✅" if player in correct_players else "❌"
        rows.append([player, guess, status])
    return rows


def build_scores_table(turn_result):
    rows = []
    scores = turn_result.get("scores", {})
    for player, score in sorted(scores.items(), key=lambda x: x[1], reverse=True):
        rows.append([player, score])
    return rows


def format_commentary(commentary: str) -> str:
    if not commentary:
        return ""
    # Add line breaks after sentence-ending punctuation for readability
    commentary = commentary.replace('. ', '.\n').replace('! ', '!\n').replace('? ', '?\n')
    return commentary


def format_hint(turn_result):
    if not turn_result:
        return ""
    domain = turn_result.get("domain", "-")
    hint = turn_result.get("hint", "")
    return f"**Domain:** {domain}\n\n**Hint:** {hint}"


def build_player_cards(turn_result, game):
    cards = []
    guesses = turn_result.get("guesses", {}) if turn_result else {}
    correct_players = set(turn_result.get("evaluation", {}).get("correct_players", [])) if turn_result else set()
    scores = turn_result.get("scores", {}) if turn_result else game.scores
    for idx, player in enumerate(game.player_models):
        shape = PLAYER_SHAPES[idx % len(PLAYER_SHAPES)]
        guess = guesses.get(player, "-")
        status = "✨✅ Correct!" if player in correct_players else "❌" if turn_result else ""
        score = scores.get(player, 0)
        card = f"{shape} **{player}**\n\nGuess: `{guess}` {status}\nScore: **{score}**"
        cards.append(card)
    return cards


In [ ]:
# Gradio UI

last_turn_result = {}


def snapshot(turn_result=None):
    tr = turn_result or last_turn_result or {}
    status_md = format_status(game)
    hint_md = format_hint(tr) if tr else "**Domain:** -\n\n**Hint:** -"
    guesses_table = build_guesses_table(tr) if tr else []
    guess_lines = build_guess_lines(tr, game)
    commentary_md = format_commentary(tr.get("commentary", "")) if tr else ""
    scores_table = build_scores_table(tr) if tr else [[p, 0] for p in game.player_models]
    winners_md = ""
    if game.game_over and getattr(game, "winners", []):
        winners_md = "\n".join([f"🏆 `{w}`" for w in game.winners])
    player_cards = build_player_cards(tr, game)
    # Ensure three cards
    while len(player_cards) < 3:
        player_cards.append("")
    return (
        status_md,
        hint_md,
        player_cards[0],
        player_cards[1],
        player_cards[2],
        guesses_table,
        guess_lines,
        commentary_md,
        scores_table,
        winners_md,
    )


def play_turn_handler():
    global last_turn_result, game
    if game.game_over:
        return (*snapshot(), gr.update(visible=False))
    last_turn_result = game.play_turn()
    # After a turn, hide the button to prevent rapid double-press
    return (*snapshot(last_turn_result), gr.update(visible=not game.game_over))


with gr.Blocks() as demo:
    gr.Markdown("## AI Trivia Contest — Gradio UI")
    gr.Markdown("Run turns to see the judge pick domains and hints while players guess in real time.")

    status_md = gr.Markdown()

    with gr.Row():
        hint_md = gr.Markdown(label="Domain & Hint")
        winners_md = gr.Markdown(label="Winners")

    gr.Markdown("### Players")
    with gr.Row():
        player1_md = gr.Markdown()
        player2_md = gr.Markdown()
        player3_md = gr.Markdown()

    gr.Markdown("### Guesses")
    guesses_tbl = gr.Dataframe(headers=["Player", "Guess", "Status"], interactive=False, wrap=True)

    gr.Markdown("### Guesses (with quick status)")
    guess_lines_md = gr.Markdown()

    gr.Markdown("### Judge Commentary")
    commentary_md = gr.Markdown()

    gr.Markdown("### Scores")
    scores_tbl = gr.Dataframe(headers=["Player", "Score"], interactive=False)

    play_btn = gr.Button("Play Next Turn", variant="primary")

    play_btn.click(
        play_turn_handler,
        inputs=[],
        outputs=[
            status_md,
            hint_md,
            player1_md,
            player2_md,
            player3_md,
            guesses_tbl,
            guess_lines_md,
            commentary_md,
            scores_tbl,
            winners_md,
            play_btn,
        ],
    )

    demo.load(
        lambda: (*snapshot(), gr.update(visible=not game.game_over)),
        inputs=[],
        outputs=[
            status_md,
            hint_md,
            player1_md,
            player2_md,
            player3_md,
            guesses_tbl,
            guess_lines_md,
            commentary_md,
            scores_tbl,
            winners_md,
            play_btn,
        ],
    )

demo.launch()


* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.


TURN 5

📚 Domain: colors
💡 Hint: "A treasure trove for the sailor who lost his way on a clear summer day."

--------------------------------------------------------------------------------
PLAYERS ARE GUESSING...
--------------------------------------------------------------------------------
  [1/3] mistral:latest is thinking... → 'blue'
  [2/3] phi3:latest is thinking... → 'azure'
  [3/3] gemma3:270m is thinking... → 'ocean'
--------------------------------------------------------------------------------
All players have guessed!

--------------------------------------------------------------------------------
GUESS RESULTS:
--------------------------------------------------------------------------------
  mistral:latest: 'blue' ✅ CORRECT!
  phi3:latest: 'azure' ❌
  gemma3:270m: 'ocean' ❌

--------------------------------------------------------------------------------
🎯 JUDGE'S COMMENTARY:
--------------------------------------------------------------------------------
  "Well, well